In [10]:
import numpy as np
import os 


DATASET_PATH = "/home/onyxia/work/hackathon-acoustic-2024/data/LivingRoom_preprocessed_hack/Human1/"

RIRs = np.load(os.path.join(DATASET_PATH, "deconvoled_trim.npy"), mmap_mode='r')
print(f"Shape RIRs : {RIRs.shape} ")
centroid = np.load(os.path.join(DATASET_PATH, "centroid.npy"), mmap_mode='r')
print(f"Shape centroid : {centroid.shape} ")

Shape RIRs : (1000, 4, 667200) 
Shape centroid : (1000, 2) 


In [12]:
def normalize_centroid(centroid):
    _max = np.array([500, 2000])
    _min = np.array([-4000, -4000])
    return 2 * ((centroid - _min) / (_max - _min)) - 1

human1_centroid_norm = normalize_centroid(centroid)

In [6]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


rir_data = np.reshape(RIRs, (1000, -1))

# Keep only the first 100 principal components
pca = PCA(n_components=100)
rir_data_pca = pca.fit_transform(rir_data)

In [18]:
rir_data_pca.shape
human1_centroid_norm.shape

(1000, 2)

In [22]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


X_train, X_test, y_train, y_test = train_test_split(rir_data_pca, human1_centroid_norm, test_size=0.2, random_state=42)

# Create KNN model with Euclidean distance metric
knn  = KNeighborsRegressor(n_neighbors=5, metric='euclidean')

# Fit the model on training data
knn.fit(X_train, y_train)

# Make predictions on test data
y_pred = knn.predict(X_test)

# Evaluate the model's performance



Accuracy: 0.7878071951784018


In [25]:
_max = np.array([500, 2000])
_min = np.array([-4000, -4000])

def unnormalize(x):
        return (x + 1) / 2 * (_max - _min) + _min
    
actual = unnormalize(y_test)
predicted = unnormalize(y_pred)

# Calculate Euclidean distance error
distance_error = np.linalg.norm(predicted - actual, axis=1)
print("Mean Distance Error in cm:", np.mean(distance_error) / 10)
print("Median Distance Error:", np.median(distance_error))
print("Maximum Distance Error:", np.max(distance_error))


Mean Distance Error in cm: 62.59923696609261
Median Distance Error: 487.1018729950647
Maximum Distance Error: 2410.9416551384993
